In [138]:
import csv
import pprint as pp
import networkx as nx
import itertools as it
import math
import scipy.sparse
import random
#Extra header files

from itertools import combinations
import numpy as np
import pandas as pd
from operator import itemgetter, attrgetter



def pagerank(M, N, nodelist, alpha=0.85, personalization=None, max_iter=100, tol=1.0e-6, dangling=None):
    if N == 0:
        return ({})
    S = scipy.array(M.sum(axis=1)).flatten()
    S[S != 0] = 1.0 / S[S != 0]
    Q = scipy.sparse.spdiags(S.T, 0, *M.shape, format='csr')
    M = Q * M
    
    # initial vector
    x = scipy.repeat(1.0 / N, N)
    
    # Personalization vector
    if personalization is None:
        p = scipy.repeat(1.0 / N, N)
    else:
        missing = set(nodelist) - set(personalization)
        if missing:
            #raise NetworkXError('Personalization vector dictionary must have a value for every node. Missing nodes %s' % missing)
            print()
            print ('Error: personalization vector dictionary must have a value for every node')
            print('inside first block')
            exit(-1)
        p = scipy.array([personalization[n] for n in nodelist], dtype=float)
        #p = p / p.sum()
        sum_of_all_components = p.sum()
        if sum_of_all_components > 1.001 or sum_of_all_components < 0.999:
            print()
            print ("Error: the personalization vector does not represent a probability distribution :(")
            print('sum_of_all')
            exit(-1)
    
    # Dangling nodes
    if dangling is None:
        dangling_weights = p
    else:
        missing = set(nodelist) - set(dangling)
        if missing:
            #raise NetworkXError('Dangling node dictionary must have a value for every node. Missing nodes %s' % missing)
            print()
            print ('Error: dangling node dictionary must have a value for every node.')
            print()
            exit(-1)
        # Convert the dangling dictionary into an array in nodelist order
        dangling_weights = scipy.array([dangling[n] for n in nodelist], dtype=float)
        dangling_weights /= dangling_weights.sum()
    is_dangling = scipy.where(S == 0)[0]

    # power iteration: make up to max_iter iterations
    for _ in range(max_iter):
        xlast = x
        x = alpha * (x * M + sum(x[is_dangling]) * dangling_weights) + (1 - alpha) * p
        # check convergence, l1 norm
        err = scipy.absolute(x - xlast).sum()
        if err < N * tol:
            return dict(zip(nodelist, map(float, x)))
    #raise NetworkXError('power iteration failed to converge in %d iterations.' % max_iter)
    print()
    print ('Error: power iteration failed to converge in '+str(max_iter)+' iterations.')
    print
    exit(-1)




def create_graph_set_of_users_set_of_items(user_item_ranking_file):
    graph_users_items = {}
    all_users_id = set()
    all_items_id = set()
    g = nx.DiGraph()
    input_file = open(user_item_ranking_file, 'r')
    input_file_csv_reader = csv.reader(input_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_NONE)
    for line in input_file_csv_reader:
        user_id = int(line[0])
        item_id = int(line[1])
        rating = int(line[2])
        g.add_edge(user_id, item_id, weight=rating)
        all_users_id.add(user_id)
        all_items_id.add(item_id)
    input_file.close()
    graph_users_items['graph'] = g
    graph_users_items['users'] = all_users_id
    graph_users_items['items'] = all_items_id
    return graph_users_items
    




def create_item_item_graph(graph_users_items):
    g = nx.Graph()
    # Your code here ;)
    gr=graph_users_items['graph']
    items_set_dct = {}
    item_list=[]
    for item in graph_users_items['items']:
        items_set_dct[item]=set()
        item_list.append(item)
        g.add_node(item)
    
    for u,v in gr.in_edges():
        items_set_dct[v].update(set([u]))

        
    for a, b in combinations(item_list, 2):
        edge_weight = len(items_set_dct[a] & items_set_dct[b])
        if edge_weight!=0:
            g.add_edge(a,b,weight = edge_weight)
    
    return g




def create_preference_vector_for_teleporting(user_id, graph_users_items):
    preference_vector = {}
    # Your code here ;)
    graph=graph_users_items['graph']
    for items in graph_users_items['items']:
        preference_vector[items]=0

    collect_sum=0
    for j in graph[user_id].keys():
        collect_sum=collect_sum+graph[user_id][j]['weight']
    
    for j in graph[user_id].keys():
        preference_vector[j]=graph[user_id][j]['weight']/collect_sum
    
    return preference_vector
    



def create_ranked_list_of_recommended_items(page_rank_vector_of_items, user_id, training_graph_users_items):
    # This is a list of 'item_id' sorted in descending order of score.
    sorted_list_of_recommended_items = []
    # You can obtain this list from a list of [item, score] couples sorted in descending order of score.
    
    # Your code here ;)
    for item in training_graph_users_items['graph'][user_id].keys():
        if item in page_rank_vector_of_items.keys():
            del page_rank_vector_of_items[item]
    
    sorted_list_of_recommended_items=sorted(page_rank_vector_of_items, key=lambda key: page_rank_vector_of_items[key],reverse=True)
    
    return sorted_list_of_recommended_items




def discounted_cumulative_gain(user_id, sorted_list_of_recommended_items, test_graph_users_items):
    dcg = 0.
    # Your code here ;)
    item_rating_list = []
    for key in test_graph_users_items['graph'][user_id].keys():
        if key in sorted_list_of_recommended_items:
            item_rating_list.append((key,test_graph_users_items['graph'][user_id][key]['weight']))
    
    zz=sorted(item_rating_list, key=lambda x: sorted_list_of_recommended_items.index(x[0]))
    c= 0
    for a,b in zz:
        c=c+1
        dcg = dcg + (b/(np.log(c+1)/np.log(2)))
        
    return dcg
    



def maximum_discounted_cumulative_gain(user_id, test_graph_users_items):
    dcg = 0.
    # Your code here ;)

    val = 5
    for i in range(len(test_graph_users_items['graph'][user_id])):
        dcg = dcg + val/(np.log(i+2)/np.log(2))
        
    return dcg



def create_preference_vector_for_teleporting_group(group_dct, graph_users_items):
    
    pref_vec_list=[]
    sum_group_val = 0
    for user_id in group_dct.keys():
        sum_group_val=sum_group_val + group_dct[user_id]
        #create the preference vector for a given user in group
        pref_vec = create_preference_vector_for_teleporting(user_id,graph_users_items)
        #weight the the ratings according to the importance of this user
        for key in pref_vec.keys():
            pref_vec[key] = pref_vec[key] * group_dct[user_id]
            
        pref_vec_list.append(pref_vec) #create a list of weighted preference vectors and then add them up
    print(sum_group_val)
    # merge these preference vectors
    merged = merge3(pref_vec_list)
    for key in merged.keys():
        merged[key]=merged[key]/sum_group_val

    
    return(merged)

    
def merge3(dicts):
    merged = {}
    for d in dicts:
        for k in d.keys():
            if k in merged.keys():
                merged[k] = merged[k] + d[k]
            else:
                merged[k]=d[k]
    return merged


def check_both_list(element_lst,seen_elements):
    for i in seen_elements:
        chk_lst=[]
        for j in i:
            chk_lst.append(j)
        if chk_lst==element_lst:
            return True
        elif chk_lst==element_lst:
            return True
    
    return False
    
from operator import itemgetter, attrgetter

def fagin_algo(tmp_text,tmp_title,k):
    seen_elements=[]
    seen_elements_both=[]
    for i,j in zip(tmp_text.movie_id,tmp_title.movie_id):
        if check_both_list([i,0,1],seen_elements)==True:
            seen_elements_both.append(i)
        if check_both_list([j,1,0],seen_elements)==True:
            seen_elements_both.append(j)
        
        #seen elements    
        seen_elements.append([i,1,0])
        seen_elements.append([j,0,1])

        if len(seen_elements_both)>=k:
            break
        
    over_all_score = []
    for elmt in seen_elements_both:
        over_all_score.append(tmp_text.loc[tmp_text.movie_id==elmt].score.values[0]+tmp_title.loc[tmp_title.movie_id==elmt].score.values[0])
    
    df = pd.DataFrame(over_all_score,seen_elements_both,columns=['score']).sort(columns='score',ascending=False)
    return list(df.index)[:k],df


In [3]:
all_groups = [
    {1701: 1, 1703: 1, 1705: 1, 1707: 1, 1709: 1}, ### Movie night with friends.
    {1701: 1, 1702: 4}, ### First appointment scenario: the preferences of the girl are 4 times more important than those of the man.
    {1701: 1, 1702: 2, 1703: 1, 1704: 2}, ### Two couples scenario: the preferences of girls are still more important than those of the men...
    {1701: 1, 1702: 1, 1703: 1, 1704: 1, 1705: 1, 1720:10}, ### Movie night with a special guest.
    {1701: 1, 1702: 1, 1703: 1, 1704: 1, 1705: 1, 1720:10, 1721:10, 1722:10}, ### Movie night with 3 special guests.
]

group_dct = {1701: 1, 1702: 4}

In [137]:
grrr=create_graph_set_of_users_set_of_items('u1_base_homework_format.txt')
item_item=create_item_item_graph(grrr)
print (" Conversion of the 'Item-Item-Graph' to a scipy sparse matrix representation.")
N = len(item_item)
nodelist = item_item.nodes()
M = nx.to_scipy_sparse_matrix(item_item, nodelist=nodelist, weight='weight', dtype=float)
print (" Done.")


 Conversion of the 'Item-Item-Graph' to a scipy sparse matrix representation.
 Done.
5


In [5]:
all_lst = []
for user_id in group_dct:
    pref_1 = create_preference_vector_for_teleporting(user_id,grrr)
    personalized_pagerank_vector_of_items = pagerank(M, N, nodelist, alpha=0.85, personalization=pref_1)
    all_lst.append(personalized_pagerank_vector_of_items)


In [17]:
df_1 = pd.DataFrame(list(all_lst[0].items()),columns=['movie_id','score'])
df_2 = pd.DataFrame(list(all_lst[1].items()),columns=['movie_id','score'])
df_1 = df_1.sort(columns='score',ascending=False)
df_2 = df_2.sort(columns='score',ascending=False)

In [29]:
#weight the pagerank scores
for i in range(df_2.shape[0]):
    df_2.iloc[i,1] = df_2.iloc[i,1]*4
    

In [135]:
#recommendation using Fagins
#Flaw: implement fagins for n lists, currently finds top k out of two lists

fagin_algo(df_1,df_2,df_1.shape[0])

/home/siddhant/python/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:35: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


([210,
  22,
  181,
  496,
  174,
  82,
  50,
  148,
  15,
  588,
  204,
  274,
  323,
  151,
  243,
  633,
  143,
  934,
  8,
  202,
  498,
  423,
  378,
  211,
  288,
  294,
  319,
  94,
  325,
  887,
  820,
  357,
  268,
  121,
  1,
  100,
  172,
  56,
  98,
  79,
  69,
  127,
  258,
  168,
  117,
  763,
  173,
  7,
  405,
  96,
  195,
  222,
  237,
  176,
  28,
  89,
  234,
  216,
  183,
  97,
  191,
  186,
  286,
  318,
  64,
  238,
  866,
  132,
  12,
  196,
  95,
  135,
  118,
  228,
  11,
  70,
  568,
  257,
  111,
  385,
  153,
  144,
  4,
  655,
  25,
  161,
  300,
  185,
  179,
  88,
  71,
  197,
  546,
  9,
  393,
  182,
  403,
  483,
  215,
  265,
  180,
  194,
  313,
  742,
  187,
  235,
  367,
  200,
  125,
  603,
  732,
  748,
  566,
  427,
  208,
  282,
  276,
  203,
  419,
  58,
  435,
  23,
  209,
  273,
  527,
  275,
  550,
  175,
  99,
  226,
  474,
  475,
  684,
  597,
  451,
  739,
  402,
  218,
  134,
  188,
  239,
  471,
  302,
  328,
  651,
  66,
  229,
  433,